In [1]:
import yaml
import numpy as np
import pandas as ps
from pathlib import Path

import matplotlib.pyplot as plt
from skimage.io import imshow
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
RANDOM_STATE = np.random.RandomState(seed=2019)
DATA = ps.read_csv(Path('..') / 'data' / 'train.csv')
DATA.head()

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,NaN
2,0002cc93b.jpg_3,NaN
3,0002cc93b.jpg_4,NaN
4,00031f466.jpg_1,NaN


In [3]:
def combine_masks(df):
    masks = [''] * 4
    for idx in df.index:
        masks[df.at[idx, 'ClassId']] = df.at[idx, 'EncodedPixels']
    return ps.Series(masks, [f'cls{i}' for i in range(1, 5)])


DATA['Image'] = DATA['ImageId_ClassId'].apply(lambda img_cls: img_cls.rsplit('_', 1)[0])
DATA['ClassId'] = DATA['ImageId_ClassId'].apply(lambda img_cls: int(img_cls.rsplit('_', 1)[1]) - 1)
# DATA = DATA[DATA['EncodedPixels'].notnull()].reset_index(drop=True)
DATA = DATA.sort_values(['Image', 'ClassId'])
DATA = DATA.groupby('Image').apply(combine_masks).reset_index()

images_folder = Path('..') / 'data' / 'train_images'
DATA['Image'] = DATA['Image'].apply(lambda img_path: (images_folder / img_path).resolve())
DATA['NumMissing'] = DATA.apply(lambda row: 4 - sum(row[f'cls{i}'] != row[f'cls{i}'] for i in range(1, 5)), axis=1)
DATA['IsAllMissing'] = DATA['NumMissing'].apply(lambda item: item == 0)


print(DATA.dtypes)

DATA.head()

Image           object
cls1            object
cls2            object
cls3            object
cls4            object
NumMissing       int64
IsAllMissing      bool
dtype: object


,Image,cls1,cls2,cls3,cls4,NumMissing,IsAllMissing
0,/home/dmdr/Documents/Code/Python/kaggle/severs...,29102 12 29346 24 29602 24 29858 24 30114 24 3...,NaN,NaN,NaN,1,False
1,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,NaN,NaN,0,True
2,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,NaN,NaN,0,True
3,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,NaN,NaN,0,True
4,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,18661 28 18863 82 19091 110 19347 110 19603 11...,NaN,1,False


In [4]:
def channels2bits(row):
    res = 0
    for i in range(1, 5):
        # not nan
        if row[f'cls{i}'] == row[f'cls{i}']:
            res = res | (1 << (i - 1))
    return res

DATA['PresentedChannelsAsBits'] = DATA.apply(channels2bits, axis=1)
DATA.head()

,Image,cls1,cls2,cls3,cls4,NumMissing,IsAllMissing,PresentedChannelsAsBits
0,/home/dmdr/Documents/Code/Python/kaggle/severs...,29102 12 29346 24 29602 24 29858 24 30114 24 3...,NaN,NaN,NaN,1,False,1
1,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,NaN,NaN,0,True,0
2,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,NaN,NaN,0,True,0
3,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,NaN,NaN,0,True,0
4,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,18661 28 18863 82 19091 110 19347 110 19603 11...,NaN,1,False,4


In [5]:
DATA['PresentedChannelsAsBits'].value_counts()

0     5902
4     4759
1      769
8      516
12     284
2      195
5       91
3       35
6       14
7        2
10       1
Name: PresentedChannelsAsBits, dtype: int64

In [6]:
DATA['PresentedChannelsAsBits'] = DATA['PresentedChannelsAsBits'].replace(6, 2 ** 4)
DATA['PresentedChannelsAsBits'] = DATA['PresentedChannelsAsBits'].replace(7, 2 ** 4)
DATA['PresentedChannelsAsBits'] = DATA['PresentedChannelsAsBits'].replace(10, 2 ** 4)
DATA['PresentedChannelsAsBits'].value_counts()

0     5902
4     4759
1      769
8      516
12     284
2      195
5       91
3       35
16      17
Name: PresentedChannelsAsBits, dtype: int64

# Segmentation

## Train & Validation split

In [7]:
def split_dataset(dataset: ps.DataFrame, 
                  validation_pcnt: float = .2,
                  random_state: np.random.RandomState = np.random.RandomState(2019),
                  how: str = 'sample',
                  stratify_set = None):
    assert how in {'sample', 'split'}, '`how` should be one of `sample` or `split` '
    
    print(f'Input shapes - {dataset.shape}')
    if how == 'sample':
        train_set = dataset.copy()

        validation_size = int(train_set.shape[0] * validation_pcnt)
        validation_set = dataset.sample(validation_size, random_state=random_state)
    else:
        train_set, validation_set = train_test_split(
            dataset, 
            test_size=validation_pcnt,
            random_state=random_state,
            stratify=stratify_set
        )
    
    print(f'Train shapes - {train_set.shape}')
    print(f'Validation shapes - {validation_set.shape}')
    return train_set, validation_set

In [8]:
train_set, validation_set = split_dataset(
    DATA[~DATA['IsAllMissing']],
    how='sample'
)

Input shapes - (6666, 8)
Train shapes - (6666, 8)
Validation shapes - (1333, 8)


In [9]:
cols = ['Image'] + [f'cls{i}' for i in range(1, 5)]

train_set = train_set[cols]
validation_set = validation_set[cols]

train_set.head()

,Image,cls1,cls2,cls3,cls4
0,/home/dmdr/Documents/Code/Python/kaggle/severs...,29102 12 29346 24 29602 24 29858 24 30114 24 3...,NaN,NaN,NaN
4,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,18661 28 18863 82 19091 110 19347 110 19603 11...,NaN
5,/home/dmdr/Documents/Code/Python/kaggle/severs...,37607 3 37858 8 38108 14 38359 20 38610 25 388...,NaN,NaN,NaN
6,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,NaN,131973 1 132228 4 132483 6 132738 8 132993 11 ...
7,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,229501 11 229741 33 229981 55 230221 77 230468...,NaN


In [10]:
split_train_set, split_validation_set = split_dataset(
    DATA[~DATA['IsAllMissing']],
    how='split',
    validation_pcnt=.15
)

Input shapes - (6666, 8)
Train shapes - (5666, 8)
Validation shapes - (1000, 8)


In [11]:
split_train_set = split_train_set[cols]
split_validation_set = split_validation_set[cols]

split_train_set.head()

,Image,cls1,cls2,cls3,cls4
11881,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,65073 7 65315 21 65558 34 65800 48 66049 55 66...,278695 8 278943 18 279196 24 279448 31 279701 ...
9305,/home/dmdr/Documents/Code/Python/kaggle/severs...,201729 26 201985 55 202241 62 202497 69 202753...,NaN,NaN,NaN
63,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,102 96 358 96 613 97 869 98 1125 98 1381 98 16...,NaN
7866,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,52225 22 52481 64 52737 107 52993 149 53249 19...,NaN
1586,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,94290 20 94544 24 94798 27 95053 30 95243 5 95...,NaN


In [12]:
raw_train_set, raw_validation_set = split_dataset(
    DATA,
    how='split',
    validation_pcnt=.1,
    stratify_set=DATA['NumMissing']
)

Input shapes - (12568, 8)
Train shapes - (11311, 8)
Validation shapes - (1257, 8)


In [13]:
raw_train_set = raw_train_set[cols]
raw_validation_set = raw_validation_set[cols]

raw_train_set.head()

,Image,cls1,cls2,cls3,cls4
1185,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,NaN,NaN
4387,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,NaN,NaN
5282,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,NaN,310279 1 310535 2 310790 4 311046 5 311097 2 3...
3204,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,201198 19 201418 55 201637 92 201857 128 20207...,NaN
164,/home/dmdr/Documents/Code/Python/kaggle/severs...,75436 41 75640 122 75863 170 76104 185 76345 2...,NaN,NaN,NaN


In [14]:
bits_train_set, bits_validation_set = split_dataset(
    DATA,
    how='split',
    validation_pcnt=.2,
    stratify_set=DATA['PresentedChannelsAsBits']
)

Input shapes - (12568, 8)
Train shapes - (10054, 8)
Validation shapes - (2514, 8)


In [15]:
bits_train_set = bits_train_set[cols]
bits_validation_set = bits_validation_set[cols]

bits_train_set.head()

,Image,cls1,cls2,cls3,cls4
11537,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,NaN,NaN
10193,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,NaN,NaN
11871,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,229121 13 229377 38 229633 62 229889 87 230145...,NaN,NaN
12421,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,NaN,NaN
6423,/home/dmdr/Documents/Code/Python/kaggle/severs...,NaN,NaN,NaN,NaN


## Dump

In [16]:
csvs_folder = Path('..') / 'data' / 'splits'

In [17]:
train_set.to_csv(csvs_folder / 'train_set.csv', index=False)
validation_set.to_csv(csvs_folder / 'validation_set.csv', index=False)

split_train_set.to_csv(csvs_folder / 'split_train_set.csv', index=False)
split_validation_set.to_csv(csvs_folder / 'split_validation_set.csv', index=False)

raw_train_set.to_csv(csvs_folder / 'raw_train_set.csv', index=False)
raw_validation_set.to_csv(csvs_folder / 'raw_validation_set.csv', index=False)

bits_train_set.to_csv(csvs_folder / 'bits_train_set.csv', index=False)
bits_validation_set.to_csv(csvs_folder / 'bits_validation_set.csv', index=False)

# Classification

In [13]:
train_set, validation_set = split_dataset(DATA, how='split', stratify_set=DATA['IsAllMissing'])

Input shapes - (12568, 7)
Train shapes - (10054, 7)
Validation shapes - (2514, 7)


In [14]:
cols = ['Image', 'IsAllMissing']

train_set = train_set[cols]
train_set['IsAllMissing'] = train_set['IsAllMissing'].astype(int)

validation_set = validation_set[cols]
validation_set['IsAllMissing'] = validation_set['IsAllMissing'].astype(int)

train_set.head()

,Image,IsAllMissing
2805,/home/dmdr/Documents/Code/Python/kaggle/severs...,1
721,/home/dmdr/Documents/Code/Python/kaggle/severs...,0
3858,/home/dmdr/Documents/Code/Python/kaggle/severs...,0
1408,/home/dmdr/Documents/Code/Python/kaggle/severs...,1
2449,/home/dmdr/Documents/Code/Python/kaggle/severs...,1


## Dump

In [15]:
train_set.to_csv(csvs_folder / 'classification_train_set.csv', index=False)
validation_set.to_csv(csvs_folder / 'classification_validation_set.csv', index=False)